In [ ]:
'''
Next, it's client to TGS - Done
Then, TGS back to client with service ticket - Done
then, client to service server - working on
Then, service server back to client - 
then, client confirms the name/id of the given service from service server - 
client to service server to give the content of the file - 
service server gives back to client with the correct content, based on the service name/id - 

'''

In [2]:
import base64
import hashlib
from cryptography.fernet import Fernet


def password_to_key(password):
    """Convert password into a Fernet encryption key (just like Kerberos derives a secret key)"""
    hash_pw = hashlib.sha256(password.encode()).digest()   # hash password
    return base64.urlsafe_b64encode(hash_pw)               # convert to valid Fernet key


In [3]:
# Simulated user database (server-side only)
database = {
    "bob": password_to_key("hello123")  # store derived key, NOT the password
}

def AuthenticationServer(username):
    if username not in database:
        return None, None   # user not found

    stored_key = database[username]
    f = Fernet(stored_key) # encryptor/decryptor

    TGT = b"TICKET_GRANTING_TICKET"
    session_key = b"SESSION_KEY_ABC123"

    encrypted_blob = f.encrypt(TGT + b"||" + session_key)
    return encrypted_blob


In [4]:
def client_login(username, password):
    # user derives key locally (password never sent)
    client_key = password_to_key(password)

    # ask server for TGT
    encrypted_reply = AuthenticationServer(username)

    if not encrypted_reply:
        print("❌ Unknown user")
        return

    try:
        f = Fernet(client_key)
        decrypted = f.decrypt(encrypted_reply)     # will fail if password wrong
        print("✅ Password correct!")
        TGT, session_key = decrypted.split(b"||")
        print("TGT =", TGT)
        print("Session Key =", session_key)

    except:
        print("❌ Wrong password! Cannot decrypt ticket.")

client_login("bob", "hello123")   # ✅ correct password
client_login("bob", "wrongpass")  # ❌ wrong password


✅ Password correct!
TGT = b'TICKET_GRANTING_TICKET'
Session Key = b'SESSION_KEY_ABC123'
❌ Wrong password! Cannot decrypt ticket.
